In [1]:
import pandas as pd
import nltk
from nltk.metrics import agreement
from nltk.metrics.agreement import AnnotationTask
from nltk.metrics import masi_distance
import math
import numpy as np
from ast import literal_eval

In [2]:
hatebr = pd.read_csv('../data/hatebr_and_rationales.csv', converters={"rationales_offensive_1_normalized": \
                                                              lambda x: x.strip('[]').replace("'", "").split(", "),
                                                             "rationales_offensive_2_normalized": \
                                                              lambda x: x.strip('[]').replace("'", "").split(", ")})

### Rationales agreements

In [5]:
rationales_1 = hatebr['rationales_offensive_1_normalized'].tolist()[:3500]

In [6]:
rationales_2 = hatebr['rationales_offensive_2_normalized'].tolist()[:3500]

#### Span-level

In [7]:
def partial_match_score(a, b, threshold=0.5):
        
    intersection= list(set(a) & set(b))
    union = list(set().union(a, b))
    iou = len(intersection)/len(union)
    return iou >= threshold
    

def jaccard_index(annot1, annot2, partial_matches=True):
    
    jaccard = []
    
    for r1, r2 in zip(annot1, annot2):
        matches = 0
        
        for span1 in r1:
            for span2 in r2:
                
                if partial_matches:
                    if partial_match_score(span1.split(' '), span2.split(' '), threshold=0.5):
                        matches+=1
                else:
                    if span1 == span2:
                        matches+=1
        
        len_unique_spans = len(r1)+len(r2)-matches
        try:
            jaccard.append(matches/len_unique_spans)
        except:
            if r1 == r2:
                jaccard.append(matches/len(r1))
        
    if partial_matches:
        print('Jaccard index span-level considering partial overlap: ', sum(jaccard)/len(annot1))
    
    else:
        print('Jaccard index span-level considering exact match: ', sum(jaccard)/len(annot1))
    

def precision(r1, r2, partial_matches=True):
    
    matches = 0
    
    for span1 in r1:
        for span2 in r2:
            
            if partial_matches:
                if partial_match_score(span1.split(' '), span2.split(' '), threshold=0.5):
                    matches += 1
            else:
                if span1 == span2:
                    matches += 1
    
    return matches/len(r1)

def recall(r1, r2, partial_matches=True):
    
    matches = 0
    
    for span1 in r1:
        for span2 in r2:
            
            if partial_matches: 
                if partial_match_score(span1.split(' '), span2.split(' '), threshold=0.5):
                    matches+=1
            
            else:
                if span1 == span2:
                    matches += 1
    
    return matches/len(r2)
    

def f1_score(annot1, annot2, partial_matches=True):
    
    prec = []
    rec = []
    f1 = []
    
    for r1, r2 in zip(annot1, annot2):
        
        p = precision(r1, r2, partial_matches)
        prec.append(p)
        r = recall(r1, r2, partial_matches)
        rec.append(r)
        
        if (p+r) > 0:
            f1.append(2*(p*r)/(p+r))
    
    if partial_matches:
    
        print('Precision span-level considering partial overlap: ', sum(prec)/len(annot1))
        print('Recall span-level considering partial overlap: ', sum(rec)/len(annot1))
        print('F1-score span-level considering partial overlap: ', sum(f1)/len(annot1))
        
    else: 
        
        print('Precision span-level considering exact match: ', sum(prec)/len(annot1))
        print('Recall span-level considering exact match: ', sum(rec)/len(annot1))
        print('F1-score span-level considering exact match: ', sum(f1)/len(annot1))

In [8]:
f1_score(rationales_1, rationales_2, partial_matches=False)
print('\n')
f1_score(rationales_2, rationales_2)

Precision span-level considering exact match:  0.4990977316334459
Recall span-level considering exact match:  0.550097959183673
F1-score span-level considering exact match:  0.5127566572107949


Precision span-level considering partial overlap:  0.7154983120054556
Recall span-level considering partial overlap:  0.7590955782312927
F1-score span-level considering partial overlap:  0.7164188005394777


In [9]:
jaccard_index(rationales_1, rationales_2, partial_matches=False)
print('\n')
jaccard_index(rationales_1, rationales_2)

['cartacapitallixo', 'cartacapitallixo'] ['cartacapitallixo', 'cartacapitallixo']
Jaccard index span-level considering exact match:  0.4692136245750898


['cartacapitallixo', 'cartacapitallixo'] ['cartacapitallixo', 'cartacapitallixo']
['um quase presidente', 'quase um apresentador'] ['um quase presidente', 'quase um apresentador']
Jaccard index span-level considering partial overlap:  0.6740386098028956


In [10]:
def f1_score_token_level(annot1, annot2):
    f1_score = []
    recs = []
    precs = []
    for r1, r2 in zip(annot1, annot2):
        
        a = ' '.join(r1).split(' ')
        b = ' '.join(r2).split(' ')
        
        precision = len(list(set(a) & set(b)))/len(a) if len(a) > 0 else 0
        precs.append(precision)
        recall = len(list(set(a) & set(b)))/len(b) if len(b) > 0 else 0
        recs.append(recall)
        
        if recall+precision > 0:
            f1_score.append(2*((recall*precision)/(recall+precision)))
    
    
    print('Precision token-level: ', sum(precs)/len(annot1))
    print('Recall token-level: ', sum(recs)/len(annot1))
    print('F1-score token-level: ', sum(f1_score)/len(annot1))
    

In [11]:
f1_score_token_level(rationales_1, rationales_2)

Precision token-level:  0.7048161268909843
Recall token-level:  0.9083967182292525
F1-score token-level:  0.7402963497883571


In [12]:
def jaccard_index_token_level(annot1, annot2):
    
    jaccard = []
    
    for r1, r2 in zip(annot1, annot2):
        
        a = ' '.join(r1).split(' ')
        b = ' '.join(r2).split(' ')
        
        intersection= list(set(a) & set(b))
        union = list(set().union(a, b))
        jaccard.append(len(intersection)/len(union))
        
    print('Jaccard index token-level: ', sum(jaccard)/len(annot1))

In [13]:
jaccard_index_token_level(rationales_1, rationales_2)

Jaccard index token-level:  0.6718647767570712


Rationales statistics

In [14]:
count = 0
terms = {}
for r in rationales_1:
    list_words = ' '.join(r).split(' ')
    count += len(list_words)
    for w in list_words:
        try:
            terms[w] += 1
        except:
            terms[w] = 1
    
    
count/len(rationales_1)

7.725142857142857

In [15]:
count = 0
terms_f = {}
for r in rationales_2:
    list_words = ' '.join(r).split(' ')
    count += len(list_words)
    for w in list_words:
        try:
            terms_f[w] += 1
        except:
            terms_f[w] = 1
    
count/len(rationales_2)

5.1414285714285715

In [16]:
count = 0
for e, r in enumerate(hatebr['normalized_text']):
    if e >= 3500:
        break
    count += len(r.split(' '))
    
count/len(rationales_1)

15.272857142857143

In [17]:
spans_r1 = []
spans_r2 = []
for r1, r2 in zip(rationales_1, rationales_2):
    for s1 in r1:
        spans_r1.append(len(s1))
    for s2 in r2:
        spans_r2.append(len(s2))

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rc 

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{amssymb}')

plt.figure()
sns.histplot(data=spans_r1, color='red', binwidth=3, element='step', label='Annotator 1', alpha=0.5)
sns.histplot(data=spans_r2, color='tab:blue', binwidth=3,  element='step', label = 'Annotator 2', alpha=0.6)
plt.ylabel('Frequency', fontsize=20)
plt.xlabel('Span Size (\# of characters)', fontsize =20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=19)
plt.savefig('histogram.jpg', bbox_inches = 'tight')
plt.show()